In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoMLOps - Batch Prediction Example

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/automlops/blob/main/examples/inference/00_batch_prediction_example.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/automlops/blob/main/examples/inference/00_batch_prediction_example.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/automlops/main/examples/inference/00_batch_prediction_example.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

# Overview

In this tutorial, you will build a [Vertex AI](https://cloud.google.com/vertex-ai) pipeline, complete with an integrated CI/CD pipeline. This tutorial will walk you through how to use AutoMLOps to define, create and run pipelines.

This tutorial assumes you have gone through the [introduction training example](../training/00_introduction_training_example.ipynb) and are using the deployed model endpoint.

# Objective
In this tutorial, you will learn how to create and run MLOps pipelines integrated with CI/CD. This tutorial goes through an example pipeline that is defined two ways: first using a custom python syntax, and second using Kubeflow syntax (either option is valid, whichever is preferred is up to you). The example pipeline runs batch prediction using a deployed classification model (see our training example for training this classification model); the pipeline runs the following component:
1. create_dataset: A custom component that will export the dataset from BQ to GCS as a jsonl.
1. batch_prediction: A custom component that will run a batch prediction job using a deployed model.

# Prerequisites

In order to use AutoMLOps, the following are required:

- Python 3.0 - 3.10
- [Google Cloud SDK 407.0.0](https://cloud.google.com/sdk/gcloud/reference)
- [beta 2022.10.21](https://cloud.google.com/sdk/gcloud/reference/beta)
- `git` installed
- `git` logged-in:
```
  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"
```
- [Application Default Credentials (ADC)](https://cloud.google.com/docs/authentication/provide-credentials-adc) are setup. This can be done through the following commands:
```
gcloud auth application-default login
gcloud config set account <account@example.com>
```

# Dependencies
- `docopt==0.6.2`,
- `docstring-parser==0.15`,
- `pipreqs==0.4.11`,
- `PyYAML==6.0.1`,
- `yarg==0.1.9`

# APIs & IAM
AutoMLOps will enable the following APIs:
- [cloudresourcemanager.googleapis.com](https://cloud.google.com/resource-manager/reference/rest)
- [aiplatform.googleapis.com](https://cloud.google.com/vertex-ai/docs/reference/rest)
- [artifactregistry.googleapis.com](https://cloud.google.com/artifact-registry/docs/reference/rest)
- [cloudbuild.googleapis.com](https://cloud.google.com/build/docs/api/reference/rest)
- [cloudscheduler.googleapis.com](https://cloud.google.com/scheduler/docs/reference/rest)
- [cloudtasks.googleapis.com](https://cloud.google.com/tasks/docs/reference/rest)
- [compute.googleapis.com](https://cloud.google.com/compute/docs/reference/rest/v1)
- [iam.googleapis.com](https://cloud.google.com/iam/docs/reference/rest)
- [iamcredentials.googleapis.com](https://cloud.google.com/iam/docs/reference/credentials/rest)
- [ml.googleapis.com](https://cloud.google.com/ai-platform/training/docs/reference/rest)
- [run.googleapis.com](https://cloud.google.com/run/docs/reference/rest)
- [storage.googleapis.com](https://cloud.google.com/storage/docs/apis)
- [sourcerepo.googleapis.com](https://cloud.google.com/source-repositories/docs/reference/rest)

AutoMLOps will update [IAM privileges](https://cloud.google.com/iam/docs/understanding-roles) for the following accounts:
1. Pipeline Runner Service Account (one is created if it does exist, defaults to: vertex-pipelines@PROJECT_ID.iam.gserviceaccount.com). Roles added:
- roles/aiplatform.user
- roles/artifactregistry.reader
- roles/bigquery.user
- roles/bigquery.dataEditor
- roles/iam.serviceAccountUser
- roles/storage.admin
- roles/run.admin
2. Cloudbuild Default Service Account (PROJECT_NUMBER@cloudbuild.gserviceaccount.com). Roles added:
- roles/run.admin
- roles/iam.serviceAccountUser
- roles/cloudtasks.enqueuer
- roles/cloudscheduler.admin

# User Guide

For a user-guide, please view these [slides](../AutoMLOps_Implementation_Guide_External.pdf).

# Costs

This tutorial uses billable components of Google Cloud:
- Vertex AI
- Artifact Registry
- Cloud Storage
- Cloud Source Repository
- Cloud Build
- Cloud Run
- Cloud Scheduler

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

# Ground-rules for using AutoMLOps
1. Do not use variables, functions, code, etc. not defined within the scope of a custom component. These custom components will become containers and will have no reference to the out of scope code.
2. Import statements and helper functions must be added inside the function. Provide parameter type hints.
3. Test each of your components for accuracy and correctness before running them using AutoMLOps. We cannot fix bugs automatically; bugs are much more difficult to fix once they are made into pipelines.
4. If you are using Kubeflow, be sure to define all the requirements needed to run the custom component - it can be easy to leave out packages which will cause the container to fail when running within a pipeline. 


# Dataset
For training data, we are using the [dry beans dataset](https://archive.ics.uci.edu/ml/datasets/dry+bean+dataset) which contains metadata on images of seven different types of dry beans taken with a high-resolution camera. The raw dataset can be found [here](https://github.com/GoogleCloudPlatform/automlops/blob/main/example/data/Dry_Beans_Dataset.csv). We will take a subset of this data to show an inferencing example.

# Setup Git
Set up your git configuration below

In [ ]:
!git config --global user.email 'you@example.com'
!git config --global user.name 'Your Name'

# Install AutoMLOps

Install AutoMLOps from [PyPI](https://pypi.org/project/google-cloud-automlops/), or locally by cloning the repo and running `pip install .`

In [ ]:
!pip3 install google-cloud-automlops --user

# Restart the kernel
Once you've installed the AutoMLOps package, you need to restart the notebook kernel so it can find the package.

**Note: Once this cell has finished running, continue on. You do not need to re-run any of the cells above.**

In [ ]:
import os

if not os.getenv('IS_TESTING'):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

# Set your project ID
Set your project ID below. If you don't know your project ID, leave the field blank and the following cells may be able to find it.

In [1]:
PROJECT_ID = '[your-project-id]'  # @param {type:"string"}

In [2]:
if PROJECT_ID == '' or PROJECT_ID is None or PROJECT_ID == '[your-project-id]':
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print('Project ID:', PROJECT_ID)

Project ID: automlops-sandbox


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


# Upload Data
This will create a BQ table and upload the Dry Beans csv. 

In [4]:
!python3 -m data.load_data_to_bq --project $PROJECT_ID --file data/Dry_Beans_Dataset_Inferencing.csv

Dataset automlops-sandbox.test_dataset already exists
Table test_dataset.dry-beans-inferencing already exists


# 1. AutoMLOps Pipeline
This workflow will define and generate a pipeline using AutoMLOps. `generate()` will create all the necessary files but not run them. `go()` will create all the necessary files, resources, push the code to the source repo to trigger the build, and then submit a Pipeline training job to Vertex AI. Please see the [readme](https://github.com/GoogleCloudPlatform/automlops/blob/main/README.md) for more information.

## Import AutoMLOps

In [5]:
from AutoMLOps import AutoMLOps

## Clear the cache
`AutoMLOps.clear_cache` will remove previous instantiations of AutoMLOps components and pipelines. Use this function if you have previously defined a component that you no longer need.

In [6]:
AutoMLOps.clear_cache()

Cache cleared.


## Batch Prediction
Define a custom component for batch prediction. Import statements and helper functions must be added inside the function.

In [7]:
@AutoMLOps.component
def batch_predict(
    project_id: str,
    bigquery_destination: str,
    bq_dataset_path: str,
    instances_format: str,
    predictions_format: str,
    model_resource_name: str,
    endpoint_resource_name: str,
    machine_type: str,
    accelerator_count: int,
    accelerator_type: str,
    max_replica_count: int,
    starting_replica_count: int
):
    """Runs a batch prediction job.

    Args:
        bigquery_destination: The BQ uri to store the prediction results.
        bq_dataset_path: The BQ uri of the input data to run predictions on.
        instances_format: The format in which instances are given, must be one of 'jsonl', 'csv', 'bigquery', 'tf-record', 'tf-record-gzip', or 'file-list'.
        predictions_format: The format to output the predictions, must be one of 'jsonl', 'csv', or 'bigquery'.
        model_resource_name: The fully-qualified resource name or ID for model e.g. projects/297370817971/locations/{region}/models/4540613586807947264
        endpoint_resource_name: The fully-qualified resource name or ID for endpoint e.g. projects/297370817971/locations/{region}/endpoints/1242430547200835584
        machine_type: The machine type to serve the prediction requests.
        accelerator_count: The number of accelerators to attach to the `machine_type`.
        accelerator_type: The type of accelerators that may be attached to the machine as per `accelerator_count`.
        max_replica_count: The maximum number of machine replicas the batch operation may be scaled to.
        starting_replica_count: The number of machine replicas used at the start of the batch operation.
    """
    import logging
    
    from google.cloud import aiplatform
    from google.cloud.aiplatform.compat.types import job_state_v1
    
    def _get_endpoint(resource_name: str) -> aiplatform.Endpoint:
        return aiplatform.Endpoint(resource_name)

    def _get_model(resource_name: str) -> aiplatform.Model:
        return aiplatform.Model(resource_name)

    def _get_model_from_endpoint(endpoint: aiplatform.Endpoint) -> aiplatform.Model:
        current_deployed_model_id = None

        traffic_split = endpoint.gca_resource.traffic_split
        for key in traffic_split:
            if traffic_split[key] == 100:
                current_deployed_model_id = key
            break

        if current_deployed_model_id:
            for deployed_model in endpoint.gca_resource.deployed_models:
                if deployed_model.id == current_deployed_model_id:
                    return aiplatform.Model(deployed_model.model)


    logging.info(f'input dataset URI: {bq_dataset_path}')

    # Call Vertex AI custom job in another region
    aiplatform.init(project=project_id)

    if model_resource_name:
        model = _get_model(model_resource_name)
    elif endpoint_resource_name:
        model = _get_model_from_endpoint(_get_endpoint(endpoint_resource_name))
    else:
        raise ValueError('model or endpoint resource name must be provided!')

    logging.info(f'retrieved model URI: {model.uri}')

    batch_pred_job = model.batch_predict(
        job_display_name='batch-prediction',
        bigquery_source=bq_dataset_path,
        bigquery_destination_prefix=bigquery_destination,
        instances_format=instances_format,
        predictions_format=predictions_format,
        machine_type=machine_type,
        accelerator_count=accelerator_count,
        accelerator_type=accelerator_type,
        starting_replica_count=starting_replica_count,
        max_replica_count=max_replica_count,
        sync=True)

    logging.info(f'batch prediction job: {batch_pred_job.resource_name}')

    batch_pred_job.wait()
    if batch_pred_job.state == job_state_v1.JobState.JOB_STATE_SUCCEEDED:
        logging.info(f'batch prediction job has finished with info: '
                     f'{batch_pred_job.completion_stats}')
        logging.info(f'Predictions can be found at: '
                     f'{batch_pred_job.output_info.gcs_output_directory}')
    else:
        raise RuntimeError(batch_pred_job.error)

## Define the Pipeline
Define your pipeline. You can optionally give the pipeline a name and description. Define the structure by listing the components to be called in your pipeline; use `.after` to specify the order of execution.

In [8]:
@AutoMLOps.pipeline #(name='automlops-pipeline', description='This is an optional description')
def pipeline(bq_table: str,
             data_path: str,
             project_id: str,
             bigquery_destination: str,
             bq_dataset_path: str,
             instances_format: str,
             predictions_format: str,
             model_resource_name: str,
             endpoint_resource_name: str,
             machine_type: str,
             accelerator_count: int,
             accelerator_type: str,
             max_replica_count: int,
             starting_replica_count: int
            ):

    batch_predict_task = batch_predict(
             project_id=project_id,
             bigquery_destination=bigquery_destination,
             bq_dataset_path=bq_dataset_path,
             instances_format=instances_format,
             predictions_format=predictions_format,
             model_resource_name=model_resource_name,
             endpoint_resource_name=endpoint_resource_name,
             machine_type=machine_type,
             accelerator_count=accelerator_count,
             accelerator_type=accelerator_type,
             max_replica_count=max_replica_count,
             starting_replica_count=starting_replica_count)

## Define the Pipeline Arguments

In [9]:
import datetime
pipeline_params = {
    'bq_table': f'{PROJECT_ID}.test_dataset.dry-beans-inferencing',
    'data_path': f'gs://{PROJECT_ID}-bucket/data.csv',
    'project_id': PROJECT_ID,
    'bigquery_destination': f'bq://{PROJECT_ID}.test_dataset.dry-beans-inferencing-results',
    'bq_dataset_path': f'bq://{PROJECT_ID}.test_dataset.dry-beans-inferencing',
    'instances_format': 'bigquery',
    'predictions_format': 'bigquery',
    'model_resource_name': '',
    'endpoint_resource_name': 'projects/45373616427/locations/us-central1/endpoints/2255296260661575680',
    'machine_type': 'n1-standard-8',
    'accelerator_count': 0,
    'accelerator_type': 'ACCELERATOR_TYPE_UNSPECIFIED',
    'max_replica_count': 2,
    'starting_replica_count': 1
}

## Generate and Run the pipeline
`AutoMLOps.generate` generates the code for the MLOps pipeline. `AutoMLOps.go` generates the code and runs the pipeline.

In [10]:
AutoMLOps.generate(project_id=PROJECT_ID,
                   pipeline_params=pipeline_params,
                   run_local=False)

INFO: Successfully saved requirements file in AutoMLOps/components/component_base/requirements.txt


In [9]:
AutoMLOps.go(project_id=PROJECT_ID,
             pipeline_params=pipeline_params,
             run_local=False)

INFO: Successfully saved requirements file in AutoMLOps/components/component_base/requirements.txt
 Updating required API services in project automlops-sandbox 
Operation "operations/acat.p2-45373616427-79a68013-c0ab-4ea0-b749-caa21aa87a80" finished successfully.
 Checking for Artifact Registry: vertex-mlops-af in project automlops-sandbox 
Listing items under project automlops-sandbox, location us-central1.

vertex-mlops-af  DOCKER  STANDARD_REPOSITORY  Artifact Registry vertex-mlops-af in us-central1.  us-central1          Google-managed key  2023-01-11T17:12:26  2023-05-15T13:15:35  40493.254
Artifact Registry: vertex-mlops-af already exists in project automlops-sandbox
 Checking for GS Bucket: automlops-sandbox-bucket in project automlops-sandbox 
gs://automlops-sandbox-bucket/
GS Bucket: automlops-sandbox-bucket already exists in project automlops-sandbox
 Checking for Service Account: vertex-pipelines in project automlops-sandbox 
Pipeline Runner Service Account         vertex-pi